# Python and databases

### Download and install the mysql 8.0 connector:
https://dev.mysql.com/downloads/connector/python/

## Documentation on the connector if you have trouble
https://dev.mysql.com/doc/connector-python/en/connector-python-installation.html

In [22]:
%config IPCompleter.greedy=True

In [23]:
## import mysql.connector
import datetime
import mysql.connector
cnx = mysql.connector.connect(user='dev', password='ax2',
                              host='127.0.0.1',
                              port='3307',
                              database='test',
                              use_pure=True) # use_pure is necessary when open_ssl package is from python 3.7 :-( 
cursor = cnx.cursor()

query = ("SELECT firstname, lastname, startdate, enddate, salary FROM pythondemo WHERE startdate BETWEEN %s AND %s")

hire_start = datetime.date(1960, 1, 1)
hire_end = datetime.date(2004, 12, 31)

cursor.execute(query, (hire_start, hire_end))

for (firstname, lastname, startdate, enddate, salary) in cursor:
  print("{} {} hired from {} to {} is paid: {} DKR pr month".format(firstname, lastname, startdate, enddate, salary))

cursor.close()
cnx.close()

Preben Kallesen hired from 1990-02-02 to 1991-01-01 is paid: 2000 DKR pr month
Hanne Jespersen hired from 1965-01-12 to 1967-02-28 is paid: 3100 DKR pr month
Karim Cuhen hired from 2001-04-11 to None is paid: 5019 DKR pr month


In [11]:
# insert, update, delete
from __future__ import print_function

from decimal import Decimal
from datetime import datetime, date, timedelta

import mysql.connector

# Connect with the MySQL Server
cnx = mysql.connector.connect(user='dev', password='ax2',
                              host='127.0.0.1',
                              port='3307',
                              database='test',
                              use_pure=True) # use_pure is necessary when open_ssl package is from python 3.7 :-( 
cursor = cnx.cursor()
# Get two buffered cursors
curA = cnx.cursor(buffered=True)
curB = cnx.cursor(buffered=True)

# Query to get employees who joined in a period defined by two dates
query = ("SELECT id, salary FROM pythondemo WHERE enddate IS NULL")

# UPDATE and INSERT statements for the old and new salary
update_old_salary = (
  "UPDATE pythondemo SET salary = %s "
  "WHERE id = %s")

# Select the employees getting a raise (all that are still employed)
curA.execute(query)

# Iterate through the result of curA
for (id, salary) in curA:
  # Update the old and insert the new salary
  new_salary = int(round(Decimal(salary) * Decimal('1.15')))
  curB.execute(update_old_salary, (new_salary, id))
  # Commit the changes
  cnx.commit()
cursor.close()
curA.close()
curB.close()
cnx.close()


In [16]:
# Fetch as dictionary
import mysql.connector
class Dict_Cursor(mysql.connector.cursor.MySQLCursor):
    def fetchone(self):
        row=self._fetch_row()
        if row:
            return dict(zip(self.column_names, row))
        return None
    
    def fetchall(self):
        result = []
        while(True):
            try:
                row = self._fetch_row()
                if(row == None):
                    break
                row = dict(zip(self.column_names, row))
                result.append(row)
            except:
                print('DONE')
        return result
            
            
            
cnx = mysql.connector.connect(user='dev', password='ax2',
                              host='127.0.0.1',
                              port='3307',
                              database='test',
                              use_pure=True) # use_pure is necessary when open_ssl package is from python 3.7 :-( 

cursor = cnx.cursor(cursor_class=Dict_Cursor)

query = ("SELECT firstname, lastname, startdate, enddate, salary FROM pythondemo")

cursor.execute(query)
showall = cursor.fetchall()
print('SHOWALL:',showall)
columns = tuple([d[0] for d in cursor.description] )
print('COLUMNS:',columns)
all = []
for (row) in cursor:
    all.append(row)
print('\nALL:',all)

cursor.close()
cnx.close()

SHOWALL: [{'firstname': 'Preben', 'lastname': 'Kallesen', 'startdate': datetime.date(1990, 2, 2), 'enddate': datetime.date(1991, 1, 1), 'salary': Decimal('2000')}, {'firstname': 'Hanne', 'lastname': 'Jespersen', 'startdate': datetime.date(1965, 1, 12), 'enddate': datetime.date(1967, 2, 28), 'salary': Decimal('3100')}, {'firstname': 'Karim', 'lastname': 'Cuhen', 'startdate': datetime.date(2001, 4, 11), 'enddate': None, 'salary': Decimal('5019')}, {'firstname': 'Aishe', 'lastname': 'Zulima', 'startdate': datetime.date(2012, 2, 18), 'enddate': None, 'salary': Decimal('6844')}]
COLUMNS: ('firstname', 'lastname', 'startdate', 'enddate', 'salary')

ALL: []


In [19]:
# with the build in MysqlCursorDict cursor
cnx = mysql.connector.connect(user='dev', password='ax2',
                              host='127.0.0.1',
                              port='3307',
                              database='test',
                              use_pure=True) # use_pure is necessary when open_ssl package is from python 3.7 :-( 

cursor = cnx.cursor(dictionary=True, buffered=True) # dict and buffered cursor

query = ("SELECT firstname, lastname, startdate, enddate, salary FROM pythondemo")

cursor.execute(query)
cursor.fetchall()

[{'firstname': 'Preben',
  'lastname': 'Kallesen',
  'startdate': datetime.date(1990, 2, 2),
  'enddate': datetime.date(1991, 1, 1),
  'salary': Decimal('2000')},
 {'firstname': 'Hanne',
  'lastname': 'Jespersen',
  'startdate': datetime.date(1965, 1, 12),
  'enddate': datetime.date(1967, 2, 28),
  'salary': Decimal('3100')},
 {'firstname': 'Karim',
  'lastname': 'Cuhen',
  'startdate': datetime.date(2001, 4, 11),
  'enddate': None,
  'salary': Decimal('5019')},
 {'firstname': 'Aishe',
  'lastname': 'Zulima',
  'startdate': datetime.date(2012, 2, 18),
  'enddate': None,
  'salary': Decimal('6844')}]

# Self study
Create a function that can take a dict and a table name and persist all values of the dict into the table columns corresponding to the dict keys.